In [10]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn import model_selection

import numpy as np

In [2]:
class Node:
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  
        self.t = t  
        self.true_branch = true_branch  
        self.false_branch = false_branch

        
class Leaf:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        classes = {}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        prediction = max(classes, key=classes.get)
        return prediction  



In [71]:
def gini(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
    return impurity

def quality(left_labels, right_labels, current_gini):
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [64]:
class ForesClassifier:
    def __init__(self, min_leaf=5):
        self.min_leaf = min_leaf
    
    def __classify_object(self, obj, node):
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.__classify_object(obj, node.true_branch)
        else:
            return self.__classify_object(obj, node.false_branch)
    
    def __split(self, data, labels, index, t):
        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]
        return true_data, false_data, true_labels, false_labels
    
    def __find_best_split(self, data, labels):
        min_leaf = self.min_leaf
        current_gini = gini(labels)
        best_quality = 0
        best_t = None
        best_index = None
        n_features = data.shape[1]

        for index in range(n_features):
            t_values = [row[index] for row in data]
            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.__split(data, labels, index, t)
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue
                current_quality = quality(true_labels, false_labels, current_gini)
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index
        return best_quality, best_t, best_index
    
    def fit(self, data, labels):
        def build_tree(data, labels):
            quality, t, index = self.__find_best_split(data, labels)
            if quality == 0:
                return Leaf(data, labels)
            true_data, false_data, true_labels, false_labels = self.__split(data, labels, index, t)
            true_branch = build_tree(true_data, true_labels)
            false_branch = build_tree(false_data, false_labels)
            return Node(index, t, true_branch, false_branch)
        self.__tree = build_tree(data, labels)

    def predict(self, data):
        classes = []
        for obj in data:
            prediction = self.__classify_object(obj, self.__tree)
            classes.append(prediction)
        return classes   
    
    def __repr__(self):
        def print_tree(node, spacing=""):
            if isinstance(node, Leaf):
                print(spacing + "Прогноз:", node.prediction)
                return
            print(spacing + 'Индекс', str(node.index))
            print(spacing + 'Порог', str(node.t))
            print (spacing + '--> True:')
            print_tree(node.true_branch, spacing + "  ")
            print (spacing + '--> False:')
            print_tree(node.false_branch, spacing + "  ")
        print_tree(self.__tree)
        return ''

In [65]:
classification_data, classification_labels = datasets.make_classification(n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

In [66]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [67]:
model = ForesClassifier()
model.fit(train_data, train_labels)

In [69]:
train_answers = model.predict(train_data)

In [70]:
print(model)

Индекс 0
Порог 0.16261402870113306
--> True:
  Индекс 1
  Порог -1.5208896621663803
  --> True:
    Индекс 0
    Порог -0.9478301462477035
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 1
  --> False:
    Прогноз: 0
--> False:
  Прогноз: 1



In [72]:
train_accuracy = accuracy_metric(train_labels, train_answers)
train_accuracy

98.57142857142858